##Read in data, apply indexs and summarise for the first time

In [1]:
#Read in data
RawDrainageData=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersDrain.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = [0,2,3], #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')

#Bring in all index data
DrainageDataIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\DrainageIndex.csv',
                         index_col = 0)

#Apply indexes to data
DrainageDataTransposed = RawDrainageData.transpose()
DrainageDataIndexed = pd.concat([DrainageDataIndex,DrainageDataTransposed], axis=1)
DrainageDataIndexed.index.name='ColumnHeader',
DrainageDataIndexed.set_index(['Measurement','Gravels','Stones','Column','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                        append=False, inplace=True)
DrainageDataIndexed.sort(inplace=True)
DrainageData=DrainageDataIndexed.transpose()
DrainageData.index = DrainageData.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back

#Set up status files
LastRow = DrainageData.index.size
np.save('DrainageLastRow.npy',LastRow)
DrainageData.to_pickle('.\DataFrames\LysoDrainage_df')

NameError: name 'pd' is not defined

##Bring in libraries

In [2]:
import datetime
import pandas as pd
import numpy as np
import plotly
import plotly.plotly as py
import plotly.tools as tls
from plotly.graph_objs import *
import time 
Tokens = tls.get_credentials_file()['stream_ids']

##Function to read in fresh data and update status files

In [3]:
def UpdataDataFrame():
    #Bring in previous status files
    LastRead = np.load('.\DrainageLastRow.npy')
    DrainageData = pd.read_pickle('.\DataFrames\LysoDrainage_df')
    
    #Bring in fresh data
    StartRead = LastRead + 4
    Skips = [0,2,3] + range(4,StartRead)
    FreshDrainageData=pd.read_csv('K:\Rainshelter\StonySoilLysimeters\DownloadedData\StonyLysimetersDrain.dat', #specify file path for data to read in
                         parse_dates=True, #tell the function to parse date columns to datetime formats
                         dayfirst=True, #tell the function that the day is before the year in the data i.e format='%d/%m/%Y %H:%M'
                         skiprows = Skips, #leave out rows 1, 3 and 4 which have redundant information
                         index_col = 0, #Use the first column, which is Date, as an index
                         na_values = 'NAN')
    #Bring in all index data
    DrainageDataIndex=pd.read_csv('C:\GitHubRepos\StonySoilLysimeters\IndexFiles\DrainageIndex.csv',
                             index_col = 0)

    #Apply indexes to data
    FreshDrainageDataTransposed = FreshDrainageData.transpose()
    FreshDrainageDataIndexed = pd.concat([DrainageDataIndex,FreshDrainageDataTransposed], axis=1)
    FreshDrainageDataIndexed.index.name='ColumnHeader',
    FreshDrainageDataIndexed.set_index(['Measurement','Gravels','Stones','Column','Sensor', 'MUX', 'Port','Units','Summary','Block','Treatment'], 
                            append=False, inplace=True)
    FreshDrainageDataIndexed.sort_index(inplace=True)
    FreshDrainageData=FreshDrainageDataIndexed.transpose()
    FreshDrainageData.index = FreshDrainageData.index.to_datetime()  ## for some reason the concat function changes the data type on the date indes so need to change it back


    #Join fresh data onto existing data
    DrainageData = pd.concat([DrainageData,FreshDrainageData])
    DrainageData.sort_index(inplace = True)
    
    #Update status files
    LastRow = DrainageData.index.size
    np.save('DrainageLastRow.npy',LastRow)
    DrainageData.to_pickle('.\DataFrames\LysoDrainage_df')
    return DrainageData

##Set up treatment description classes

In [4]:
#Treatment class constructor
class Treatment(object):
    def __init__(self, Number, Depth, Stonyness):
        self.Number=Number
        self.Depth=Depth
        self.Stonyness=Stonyness
        self.streamID=Tokens[number+12] #First 6 takens used by Rainshelter graph, lysimeter uses the 2nd set
        self.Label= Depth + ' ' + Stonyness
        self.Trace = Scatter(x=[],y=[], name = self.Label,
                stream = Stream(token=self.streamID)
                )
        self.Stream = py.Stream(self.streamID)

#Creat class with treatment data
Treatments = []
Colors = ['blue','green','red']
Depths = ['Deep', 'Shallow']
Stones = ['StoneFree', 'Stony', 'VeryStony']
Lines = ['solid','dash']
number = 0
LinePos = 0
for D in Depths:
    ColorPos = 0
    for S in Stones:
        Treatments.append(Treatment(number, D, S))
        Treatments[number].Trace.line = Line(color = Colors[ColorPos], dash = Lines[LinePos])
        ColorPos +=1
        number += 1
    LinePos+=1

In [5]:
for T in Treatments:
    print(T.streamID)

4e70vk8kvi
ezvz3b2q3c
i40kblx3d2
z59gf97khg
rq4a7mcfzs
ljfobancoa


##Build base graph

In [6]:
GraphData = Data([Treatments[0].Trace,
                  Treatments[1].Trace,
                  Treatments[2].Trace,
                  Treatments[3].Trace,
                  Treatments[4].Trace,
                  Treatments[5].Trace
                 ])

layout = Layout(title='Lysimeter Drainage flux',
                xaxis=XAxis(title='Date'),
                yaxis=YAxis(title='drainage (tips/h)'),
                autosize=True)

Fig = Figure(data=GraphData, layout=layout)

unique_url = py.iplot(Fig, filename = 'LysimeterDrainage')

##Function to update plotly graph

In [7]:
def Update():
    #Update Data Frame with data that has been logged since last update
    FieldData = UpdataDataFrame()
    
    #Calculate treatment means omitting data prior to 2014-11-05 08:00:00 to avoid NaN values
    DataSums =  FieldData.ix['2015-06-01 08:00:00':,'Drainage'].groupby(level=['Gravels','Stones'],axis=1).mean()
    DataSums =  DataSums.dropna(axis=1) #For some reason it keeps non valid combinations in so need to extract with this function
    DataSums = DataSums.resample('H1', how='sum')
    
    X = DataSums.index
    for Treatment in Treatments:
        Treatment.Stream.open()
        Treatment.Stream.write({'x':X,'y':DataSums.ix[:,(Treatment.Depth,Treatment.Stonyness)]})
        Treatment.Stream.close()

##One off update

In [66]:
Update()

##Continious update

In [ ]:
while True:
    try:
        Update() #get fresh data and send to graph
    except:
        retrys = 1
        print('Attempt to stream data failed')
        while retrys < 5: ##retry connecting 4 times
            time.sleep(160)  ##wait 2 min 
            try:
                Update() # have another go
            except:
                print('Retry' + str(retrys) + ' failed')
                retrys +=1 #increment try counter and try again
            else:
                retrys = 6 #set try counter so loop is exited
    time.sleep(3600)  ## update again in an hours time